In [32]:
import sys
import math
from pathlib import Path
from typing import Optional, Tuple

import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd
from jaxopt import LBFGS

In [33]:
def _read_samples(path: str | Path) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """
    Legge il CSV dell’istogramma.
    Returns
    -------
    freq : jnp.ndarray shape (num_conf,)
    configs : jnp.ndarray shape (num_conf, num_spins) valori ±1
    """
    df = pd.read_csv(path, header=None)
    arr = df.values
    freq = arr[:, 0].astype(np.float32)
    configs = arr[:, 1:].astype(np.float32)
    return jnp.asarray(freq), jnp.asarray(configs)

In [34]:
def _read_adjacency(path: str | Path, n: int) -> jnp.ndarray:
    """Legge la matrice di adiacenza opzionale."""
    df = pd.read_csv(path, header=None)
    adj = df.values.astype(np.float32)
    if adj.shape != (n, n):
        raise ValueError(f"Adjacency must be {n}×{n}, got {adj.shape}")
    return jnp.asarray(adj)

In [35]:
def _compute_lambda(alpha: float, n_spins: int, n_samples: int) -> float:
    """Stessa formula dello script Julia."""
    return alpha * math.sqrt(math.log((n_spins**2) / 0.05) / n_samples)


In [36]:
def _rise_loss(h: jnp.ndarray) -> jnp.ndarray:
    return jnp.exp(-h)


def _logrise_loss(h: jnp.ndarray) -> jnp.ndarray:
    # NB: la parte log viene messa nel caller
    return jnp.exp(-h)


def _rple_loss(h: jnp.ndarray) -> jnp.ndarray:
    return jnp.log1p(jnp.exp(-2.0 * h))  # log(1 + e^{-2h})


In [37]:
from jaxopt import LBFGS
import jax.numpy as jnp
from typing import Optional

# … (resto degli import e delle funzioni d'appoggio) …

def _reconstruct_single_spin(
    s: int,
    freq: jnp.ndarray,
    configs: jnp.ndarray,
    method: str,
    lam: float,
    adj_row: Optional[jnp.ndarray],
) -> jnp.ndarray:
    """
    Ricostruisce i pesi w_{s,i} per il singolo spin s.
    Restituisce un vettore di lunghezza num_spins.
    """
    num_conf, num_spins = configs.shape
    n_samples = freq.sum()

    # ---------- statistiche nodali ----------
    y = configs[:, s]
    nodal_stat = y[:, None] * configs          # y * x_i
    nodal_stat = nodal_stat.at[:, s].set(y)    # colonna del campo
    nodal_stat = nodal_stat.astype(jnp.float32)

    # ---------- maschere ----------
    l1_mask = jnp.ones(num_spins, dtype=jnp.float32).at[s].set(0.0)
    zero_mask = (
        (adj_row == 0) & (jnp.arange(num_spins) != s)
        if adj_row is not None
        else jnp.zeros(num_spins, dtype=bool)
    )

    free_idx = jnp.where(~zero_mask)[0]              # indici liberi
    l1_mask_free = l1_mask[free_idx]

    # ---------- loss smooth ----------
    def loss_smooth(w_free):
        w_full = jnp.zeros(num_spins, dtype=jnp.float32).at[free_idx].set(w_free)
        h = nodal_stat @ w_full
        if method == "RISE":
            return (freq / n_samples * jnp.exp(-h)).sum()
        elif method == "logRISE":
            return jnp.log((freq / n_samples * jnp.exp(-h)).sum())
        elif method == "RPLE":
            return (freq / n_samples * jnp.log1p(jnp.exp(-2.0 * h))).sum()
        else:
            raise ValueError(f"Metodo non riconosciuto: {method}")

    # ---------- loss totale ----------
    def objective(w_free):
        return loss_smooth(w_free) + lam * jnp.sum(l1_mask_free * jnp.abs(w_free))

    # ---------- ottimizzazione LBFGS ----------
    init_w = jnp.zeros((free_idx.size,), dtype=jnp.float32)
    solver = LBFGS(fun=objective, maxiter=500, tol=1e-6)
    sol = solver.run(init_w)
    w_opt_free = sol.params                              # ← ora definito

    # ---------- reinserimento ----------
    w_full = jnp.zeros(num_spins, dtype=jnp.float32).at[free_idx].set(w_opt_free)
    return w_full


In [38]:
def inverse_ising(
    method: str,
    regularizing_value: float,
    symmetrization: str,
    file_samples_histo: str | Path,
    file_reconstruction: str | Path = "reconstruction.csv",
    adjacency_path: Optional[str | Path] = None,
) -> np.ndarray:
    """
    Ricostruisce la matrice dei parametri Ising.
    Restituisce `W` (numpy array) e salva su CSV se richiesto.
    """
    method = method.strip()
    symmetrization = symmetrization.strip().upper()

    # ---- lettura dati ----
    freq, configs = _read_samples(file_samples_histo)
    num_conf, num_spins = configs.shape
    num_samples = float(freq.sum())

    adj = None
    if adjacency_path is not None:
        adj = _read_adjacency(adjacency_path, num_spins)

    lam = _compute_lambda(regularizing_value, num_spins, num_samples)
    print(f"λ = {lam:.5g}  (reg = {regularizing_value})")

    # ---- ricostruzione spin per spin ----
    rows = []
    for s in range(num_spins):
        print(f"[{s+1}/{num_spins}] Ricostruzione spin {s}")
        adj_row = adj[s] if adj is not None else None
        w_row = _reconstruct_single_spin(
            s, freq, configs, method, lam, adj_row
        )
        rows.append(w_row)

    # ---- matrice finale ----
    W = jnp.stack(rows)  # (n, n)

    if symmetrization == "Y":
        W = 0.5 * (W + W.T)

    W_np = np.asarray(W)

    # ---- salvataggio ----
    pd.DataFrame(W_np).to_csv(file_reconstruction, header=False, index=False)
    print(f"✔ Matrice salvata in '{file_reconstruction}'")

    return W_np

In [39]:
# ⬇️ Cell to run ⬇️
import pandas as pd
from pathlib import Path

# ────────────────────────────────────────────────
# ♦ CONFIGURA QUI I PARAMETRI ♦
method              = "RPLE"      # "RISE", "logRISE" o "RPLE"
regularizing_value  = 0.2         # coefficiente α (0 < α < 1 di solito)
symmetrization      = "Y"         # "Y" = simmetrizza; "N" = lascia asimmetrico
file_samples_histo  = "output_samples.csv"   # CSV con [freq, spin1, spin2, ...]
file_reconstruction = "reconstruction.csv"   # dove salvare la matrice stimata
adjacency_path      = None   # None se nessun vincolo
# ────────────────────────────────────────────────

# (controllo rapido percorsi)
file_samples_histo = Path(file_samples_histo)
if not file_samples_histo.exists():
    raise FileNotFoundError(f"{file_samples_histo} non trovato")

if adjacency_path is not None:
    adjacency_path = Path(adjacency_path)
    if not adjacency_path.exists():
        raise FileNotFoundError(f"{adjacency_path} non trovato")

print("► Parametri impostati manualmente:")
print(f"  method              = {method}")
print(f"  regularizing_value  = {regularizing_value}")
print(f"  symmetrization      = {symmetrization}")
print(f"  file_samples_histo  = {file_samples_histo}")
print(f"  file_reconstruction = {file_reconstruction}")
print(f"  adjacency_path      = {adjacency_path}")

# ---- chiamata a inverse_ising ----
W = inverse_ising(
    method=method,
    regularizing_value=regularizing_value,
    symmetrization=symmetrization,
    file_samples_histo=file_samples_histo,
    file_reconstruction=file_reconstruction,
    adjacency_path=adjacency_path,
)

print("► Ricostruzione completata. Matrice W:")
W  # Jupyter mostrerà la matrice


► Parametri impostati manualmente:
  method              = RPLE
  regularizing_value  = 0.2
  symmetrization      = Y
  file_samples_histo  = output_samples.csv
  file_reconstruction = reconstruction.csv
  adjacency_path      = None
λ = 0.0005437  (reg = 0.2)
[1/9] Ricostruzione spin 0
[2/9] Ricostruzione spin 1
[3/9] Ricostruzione spin 2
[4/9] Ricostruzione spin 3
[5/9] Ricostruzione spin 4
[6/9] Ricostruzione spin 5
[7/9] Ricostruzione spin 6
[8/9] Ricostruzione spin 7
[9/9] Ricostruzione spin 8
✔ Matrice salvata in 'reconstruction.csv'
► Ricostruzione completata. Matrice W:


array([[ 2.5030889e-03, -1.8987954e-01,  4.9384075e-01,  4.9586788e-01,
         1.0000688e-05,  1.0677764e-05,  4.9714506e-01, -6.8499588e-07,
         3.5643796e-03],
       [-1.8987954e-01, -3.9753562e-01,  4.9283773e-01,  5.1012859e-07,
         4.9524781e-01,  1.4128968e-05,  2.2624012e-03,  4.9776053e-01,
        -5.1833067e-06],
       [ 4.9384075e-01,  4.9283773e-01, -7.3066261e-04,  9.6953132e-05,
         1.7073794e-03,  4.9782211e-01, -1.7276859e-06,  3.1087748e-09,
         4.9818593e-01],
       [ 4.9586788e-01,  5.1012859e-07,  9.6953132e-05,  2.9886478e-01,
         4.9933013e-01,  4.9733964e-01,  4.9903485e-01,  2.3163240e-03,
        -1.5785038e-06],
       [ 1.0000688e-05,  4.9524781e-01,  1.7073794e-03,  4.9933013e-01,
        -4.1894871e-03,  4.9422765e-01,  3.5107406e-04,  4.9587715e-01,
         1.5130081e-03],
       [ 1.0677764e-05,  1.4128968e-05,  4.9782211e-01,  4.9733964e-01,
         4.9422765e-01, -2.4973792e-03,  1.6412610e-04,  2.1629622e-03,
         5.